### import libraries

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, SimpleRNN, TimeDistributed, LSTM, GRU, Bidirectional
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

### Data load

In [4]:
Data = np.load('DATA/EpilepsyData.npz', allow_pickle=True)

In [5]:
Train_x = Data['Train_x']
Train_y = Data['Train_y']
Test_x = Data['Test_x']
Test_y = Data['Test_y']

In [6]:
Train_x

array([[[ 0.35,  0.22,  0.55],
        [ 0.4 ,  0.01,  0.51],
        [ 0.44, -0.71,  0.52],
        ...,
        [ 0.62,  0.72,  0.5 ],
        [ 0.65, -0.9 ,  0.55],
        [ 0.69, -1.3 ,  0.54]],

       [[-0.91, -0.05,  0.06],
        [-0.92, -0.02, -1.57],
        [-0.95,  0.01, -0.99],
        ...,
        [-0.47,  0.88, -0.95],
        [-0.45,  0.8 , -0.83],
        [-0.45,  0.76, -0.78]],

       [[ 0.32,  1.  ,  0.37],
        [ 0.71, -1.33,  0.37],
        [ 0.7 , -1.44,  0.37],
        ...,
        [ 0.59, -1.04,  0.45],
        [ 0.59,  0.47, -0.64],
        [ 0.59,  1.23,  0.66]],

       ...,

       [[ 1.16,  0.9 , -1.03],
        [-0.02,  0.59, -1.11],
        [-1.04,  0.52, -0.88],
        ...,
        [-0.51,  0.29, -0.02],
        [-0.94,  0.29, -0.02],
        [-0.48,  0.29, -0.02]],

       [[-0.74, -0.19, -0.14],
        [-0.14, -0.24, -0.26],
        [ 0.12, -0.34, -0.27],
        ...,
        [-0.4 , -0.54, -0.45],
        [-0.23, -0.43, -0.86],
        [-0.27,

In [7]:
Train_x.dtype

dtype('float64')

In [8]:
Train_x.shape

(137, 206, 3)

In [9]:
TimeRange = Train_x.shape[1]
nFeatures = Train_x.shape[2]

### Data Shuffle

In [10]:
arr = np.arange(Train_x.shape[0])
np.random.shuffle(arr)
arr

array([134,  71,  77, 104,  54,  21,  40,  43,  73,  12, 113,  39,  75,
        32, 129,  49,  82, 121,  25, 109,  89,   2,   3,  76,   1,  95,
        78,  94,  91, 118,  42,  52,  56,  92, 112,  85, 111,  36,  16,
        18,  34,  29,  66,  84,  74, 133,  28,  67,  47,  17,   4,  93,
       132,  81,  31,  48,  50,  10,  60, 135, 105, 101, 106,  59,  98,
        83, 125,  90,  27, 136,  72,  20,  53,  55,  45,  23,  63, 128,
        96,   0,  51,  30, 130,  38, 120,  14, 126,  64, 124,  61,  88,
       110, 117, 119,  13, 131, 115,   9,   5,  44,  68, 127,  22, 107,
        26,  41, 122,  15, 123, 116,  99,  65,  35,  86,  24,   7,  79,
        69, 100,  19,  80,   8,  33, 102,  57, 103, 108,   6,  58,  97,
        37, 114,  46,  87,  62,  11,  70])

In [11]:
Train_x = Train_x[arr]
Train_y = Train_y[arr]

In [12]:
arr = np.arange(Test_x.shape[0])
np.random.shuffle(arr)

Test_x = Test_x[arr]
Test_y = Test_y[arr]

### Model structure with Simple RNN

In [13]:
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(SimpleRNN(24, activation='linear', return_sequences = True, input_shape=( TimeRange, nFeatures,  )))
model.add(SimpleRNN(24, activation='linear', return_sequences = False, input_shape=( TimeRange, nFeatures,  )))
model.add(Dense(4, activation='softmax'))

In [14]:
model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 206, 24)           672       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 24)                1176      
                                                                 
 dense (Dense)               (None, 4)                 100       
                                                                 
Total params: 1,948
Trainable params: 1,948
Non-trainable params: 0
_________________________________________________________________


### Call-back function for fine-tunning

In [15]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='MODEL/{epoch:03d}-{loss:.4f}-{val_loss:.4f}.hdf5',
            monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#minitor -> loss -> val_loss

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, min_lr=1e-8)
# factor: Learning rate에 곱할 것.
#0.1 -> 0.08 -> 0.064 ....
#monitor='loss' -> monitor='val_loss'

CALLBACK = [CP, LR]

In [14]:
history_finetunning = model.fit(
    Train_x, Train_y,
    epochs=300,
    batch_size=52,
    callbacks=CALLBACK,
    validation_data=(Test_x, Test_y),
    shuffle=True)

Epoch 1/300
3/3 [==============================] - ETA: 0s - loss: 53.4575 - accuracy: 0.2701 
Epoch 1: val_loss improved from inf to 16.68031, saving model to MODEL\001-53.4575-16.6803.hdf5
3/3 [==============================] - 1s 169ms/step - loss: 53.4575 - accuracy: 0.2701 - val_loss: 16.6803 - val_accuracy: 0.2464 - lr: 0.0010
Epoch 2/300
3/3 [==============================] - ETA: 0s - loss: 12.7629 - accuracy: 0.2847
Epoch 2: val_loss improved from 16.68031 to 11.13915, saving model to MODEL\002-12.7629-11.1391.hdf5
3/3 [==============================] - 0s 52ms/step - loss: 12.7629 - accuracy: 0.2847 - val_loss: 11.1391 - val_accuracy: 0.1957 - lr: 0.0010
Epoch 3/300
3/3 [==============================] - ETA: 0s - loss: 8.3742 - accuracy: 0.2993
Epoch 3: val_loss improved from 11.13915 to 8.78602, saving model to MODEL\003-8.3742-8.7860.hdf5
3/3 [==============================] - 0s 52ms/step - loss: 8.3742 - accuracy: 0.2993 - val_loss: 8.7860 - val_accuracy: 0.1667 - lr: 0.

Epoch 26/300
3/3 [==============================] - ETA: 0s - loss: 2.0035 - accuracy: 0.3285
Epoch 26: val_loss did not improve from 3.33177
3/3 [==============================] - 0s 46ms/step - loss: 2.0035 - accuracy: 0.3285 - val_loss: 3.3622 - val_accuracy: 0.2899 - lr: 0.0010
Epoch 27/300
3/3 [==============================] - ETA: 0s - loss: 1.9711 - accuracy: 0.3504
Epoch 27: val_loss did not improve from 3.33177
3/3 [==============================] - 0s 47ms/step - loss: 1.9711 - accuracy: 0.3504 - val_loss: 3.3411 - val_accuracy: 0.2971 - lr: 0.0010
Epoch 28/300
3/3 [==============================] - ETA: 0s - loss: 1.9351 - accuracy: 0.3504
Epoch 28: val_loss improved from 3.33177 to 3.30949, saving model to MODEL\028-1.9351-3.3095.hdf5
3/3 [==============================] - 0s 54ms/step - loss: 1.9351 - accuracy: 0.3504 - val_loss: 3.3095 - val_accuracy: 0.2971 - lr: 0.0010
Epoch 29/300
3/3 [==============================] - ETA: 0s - loss: 1.9106 - accuracy: 0.3285
Epoch 2

3/3 [==============================] - ETA: 0s - loss: 1.2709 - accuracy: 0.4745
Epoch 53: val_loss improved from 2.95904 to 2.95637, saving model to MODEL\053-1.2709-2.9564.hdf5
3/3 [==============================] - 0s 53ms/step - loss: 1.2709 - accuracy: 0.4745 - val_loss: 2.9564 - val_accuracy: 0.3188 - lr: 0.0010
Epoch 54/300
3/3 [==============================] - ETA: 0s - loss: 1.2604 - accuracy: 0.4599
Epoch 54: val_loss improved from 2.95637 to 2.93943, saving model to MODEL\054-1.2604-2.9394.hdf5
3/3 [==============================] - 0s 55ms/step - loss: 1.2604 - accuracy: 0.4599 - val_loss: 2.9394 - val_accuracy: 0.3116 - lr: 0.0010
Epoch 55/300
3/3 [==============================] - ETA: 0s - loss: 1.2423 - accuracy: 0.4599
Epoch 55: val_loss improved from 2.93943 to 2.92465, saving model to MODEL\055-1.2423-2.9247.hdf5
3/3 [==============================] - 0s 54ms/step - loss: 1.2423 - accuracy: 0.4599 - val_loss: 2.9247 - val_accuracy: 0.3116 - lr: 0.0010
Epoch 56/300
3

3/3 [==============================] - ETA: 0s - loss: 0.8062 - accuracy: 0.6934
Epoch 107: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 45ms/step - loss: 0.8062 - accuracy: 0.6934 - val_loss: 2.7452 - val_accuracy: 0.4058 - lr: 6.4000e-04
Epoch 108/300
3/3 [==============================] - ETA: 0s - loss: 0.8107 - accuracy: 0.7153
Epoch 108: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 44ms/step - loss: 0.8107 - accuracy: 0.7153 - val_loss: 2.7338 - val_accuracy: 0.3768 - lr: 6.4000e-04
Epoch 109/300
3/3 [==============================] - ETA: 0s - loss: 0.7977 - accuracy: 0.7080
Epoch 109: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 45ms/step - loss: 0.7977 - accuracy: 0.7080 - val_loss: 2.6612 - val_accuracy: 0.4130 - lr: 6.4000e-04
Epoch 110/300
3/3 [==============================] - ETA: 0s - loss: 0.7865 - accuracy: 0.7080
Epoch 110: val_loss did not improve from 2.60666
3/3

Epoch 135/300
3/3 [==============================] - ETA: 0s - loss: 0.6892 - accuracy: 0.7445
Epoch 135: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 49ms/step - loss: 0.6892 - accuracy: 0.7445 - val_loss: 2.9052 - val_accuracy: 0.4130 - lr: 3.2768e-04
Epoch 136/300
3/3 [==============================] - ETA: 0s - loss: 0.6880 - accuracy: 0.7372
Epoch 136: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 45ms/step - loss: 0.6880 - accuracy: 0.7372 - val_loss: 2.9360 - val_accuracy: 0.4130 - lr: 3.2768e-04
Epoch 137/300
3/3 [==============================] - ETA: 0s - loss: 0.6814 - accuracy: 0.7445
Epoch 137: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 46ms/step - loss: 0.6814 - accuracy: 0.7445 - val_loss: 2.9526 - val_accuracy: 0.3841 - lr: 3.2768e-04
Epoch 138/300
3/3 [==============================] - ETA: 0s - loss: 0.6848 - accuracy: 0.7445
Epoch 138: val_loss did not improve fr

Epoch 163/300
3/3 [==============================] - ETA: 0s - loss: 0.6300 - accuracy: 0.7810
Epoch 163: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 47ms/step - loss: 0.6300 - accuracy: 0.7810 - val_loss: 3.1303 - val_accuracy: 0.4058 - lr: 1.6777e-04
Epoch 164/300
3/3 [==============================] - ETA: 0s - loss: 0.6289 - accuracy: 0.7883
Epoch 164: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 46ms/step - loss: 0.6289 - accuracy: 0.7883 - val_loss: 3.1367 - val_accuracy: 0.3986 - lr: 1.6777e-04
Epoch 165/300
3/3 [==============================] - ETA: 0s - loss: 0.6343 - accuracy: 0.7810
Epoch 165: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 46ms/step - loss: 0.6343 - accuracy: 0.7810 - val_loss: 3.1090 - val_accuracy: 0.4058 - lr: 1.6777e-04
Epoch 166/300
3/3 [==============================] - ETA: 0s - loss: 0.6242 - accuracy: 0.7737
Epoch 166: val_loss did not improve fr

Epoch 191/300
3/3 [==============================] - ETA: 0s - loss: 0.6001 - accuracy: 0.7737
Epoch 191: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 45ms/step - loss: 0.6001 - accuracy: 0.7737 - val_loss: 3.1922 - val_accuracy: 0.3913 - lr: 1.0737e-04
Epoch 192/300
3/3 [==============================] - ETA: 0s - loss: 0.5992 - accuracy: 0.7956
Epoch 192: val_loss did not improve from 2.60666

Epoch 192: ReduceLROnPlateau reducing learning rate to 8.589935605414213e-05.
3/3 [==============================] - 0s 46ms/step - loss: 0.5992 - accuracy: 0.7956 - val_loss: 3.1977 - val_accuracy: 0.3986 - lr: 1.0737e-04
Epoch 193/300
3/3 [==============================] - ETA: 0s - loss: 0.5983 - accuracy: 0.7956
Epoch 193: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 45ms/step - loss: 0.5983 - accuracy: 0.7956 - val_loss: 3.1981 - val_accuracy: 0.3986 - lr: 8.5899e-05
Epoch 194/300
3/3 [==============================] - E

Epoch 219/300
3/3 [==============================] - ETA: 0s - loss: 0.5832 - accuracy: 0.7737
Epoch 219: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 46ms/step - loss: 0.5832 - accuracy: 0.7737 - val_loss: 3.2725 - val_accuracy: 0.3913 - lr: 5.4976e-05
Epoch 220/300
3/3 [==============================] - ETA: 0s - loss: 0.5818 - accuracy: 0.7810
Epoch 220: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 45ms/step - loss: 0.5818 - accuracy: 0.7810 - val_loss: 3.2790 - val_accuracy: 0.3913 - lr: 5.4976e-05
Epoch 221/300
3/3 [==============================] - ETA: 0s - loss: 0.5823 - accuracy: 0.7883
Epoch 221: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 46ms/step - loss: 0.5823 - accuracy: 0.7883 - val_loss: 3.2819 - val_accuracy: 0.3913 - lr: 5.4976e-05
Epoch 222/300
3/3 [==============================] - ETA: 0s - loss: 0.5819 - accuracy: 0.7956
Epoch 222: val_loss did not improve fr

Epoch 247/300
3/3 [==============================] - ETA: 0s - loss: 0.5738 - accuracy: 0.8102
Epoch 247: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 48ms/step - loss: 0.5738 - accuracy: 0.8102 - val_loss: 3.3041 - val_accuracy: 0.3913 - lr: 2.8147e-05
Epoch 248/300
3/3 [==============================] - ETA: 0s - loss: 0.5733 - accuracy: 0.8029
Epoch 248: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 49ms/step - loss: 0.5733 - accuracy: 0.8029 - val_loss: 3.3068 - val_accuracy: 0.3913 - lr: 2.8147e-05
Epoch 249/300
3/3 [==============================] - ETA: 0s - loss: 0.5734 - accuracy: 0.8175
Epoch 249: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 51ms/step - loss: 0.5734 - accuracy: 0.8175 - val_loss: 3.3089 - val_accuracy: 0.3913 - lr: 2.8147e-05
Epoch 250/300
3/3 [==============================] - ETA: 0s - loss: 0.5727 - accuracy: 0.8175
Epoch 250: val_loss did not improve fr

Epoch 275/300
3/3 [==============================] - ETA: 0s - loss: 0.5684 - accuracy: 0.8175
Epoch 275: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 45ms/step - loss: 0.5684 - accuracy: 0.8175 - val_loss: 3.3240 - val_accuracy: 0.3841 - lr: 1.4412e-05
Epoch 276/300
3/3 [==============================] - ETA: 0s - loss: 0.5684 - accuracy: 0.8175
Epoch 276: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 45ms/step - loss: 0.5684 - accuracy: 0.8175 - val_loss: 3.3250 - val_accuracy: 0.3841 - lr: 1.4412e-05
Epoch 277/300
3/3 [==============================] - ETA: 0s - loss: 0.5684 - accuracy: 0.8175
Epoch 277: val_loss did not improve from 2.60666
3/3 [==============================] - 0s 48ms/step - loss: 0.5684 - accuracy: 0.8175 - val_loss: 3.3256 - val_accuracy: 0.3841 - lr: 1.4412e-05
Epoch 278/300
3/3 [==============================] - ETA: 0s - loss: 0.5680 - accuracy: 0.8175
Epoch 278: val_loss did not improve fr

In [15]:
res = model.predict(Test_x)

5/5 [==============================] - 0s 7ms/step


In [16]:
res.argmax(axis=1)

array([3, 3, 3, 2, 2, 1, 0, 2, 0, 2, 2, 2, 3, 2, 1, 1, 3, 1, 2, 3, 3, 3,
       2, 3, 2, 3, 3, 2, 0, 1, 1, 0, 2, 0, 2, 2, 3, 0, 3, 3, 3, 3, 0, 0,
       1, 0, 2, 1, 3, 2, 3, 2, 3, 0, 3, 3, 3, 0, 3, 2, 1, 0, 3, 3, 1, 2,
       2, 0, 2, 3, 0, 1, 3, 3, 2, 3, 1, 2, 3, 3, 2, 3, 2, 3, 3, 2, 0, 3,
       0, 3, 0, 3, 2, 2, 1, 3, 3, 3, 3, 3, 1, 1, 3, 3, 0, 3, 3, 3, 3, 2,
       1, 2, 3, 2, 2, 1, 1, 3, 2, 3, 3, 1, 3, 2, 1, 2, 3, 3, 1, 3, 2, 3,
       3, 1, 3, 2, 3, 0], dtype=int64)

In [17]:
Test_y.argmax(axis=1)

array([1, 3, 3, 0, 0, 0, 2, 0, 0, 3, 1, 2, 3, 1, 1, 2, 3, 1, 2, 3, 2, 0,
       3, 3, 3, 3, 1, 2, 0, 1, 1, 1, 2, 3, 2, 2, 0, 0, 3, 2, 3, 2, 1, 3,
       3, 0, 2, 1, 2, 0, 3, 2, 3, 0, 2, 1, 1, 3, 0, 3, 1, 0, 0, 1, 0, 0,
       0, 0, 2, 1, 0, 3, 3, 0, 2, 1, 3, 0, 1, 1, 2, 1, 2, 3, 2, 0, 2, 1,
       2, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 3, 3, 2, 3, 3, 3, 1, 2, 1, 1, 0,
       0, 3, 3, 2, 1, 0, 0, 1, 2, 0, 0, 3, 3, 0, 3, 2, 3, 1, 1, 1, 2, 2,
       3, 2, 3, 0, 3, 1], dtype=int64)

In [20]:
pd.Series(res.argmax(axis=1) == Test_y.argmax(axis=1)).value_counts()

False    84
True     54
dtype: int64

# LSTM
### lstm long short term memory 

In [16]:
model = Sequential()

# RNN. 현재 RNN은 return_sequences (모든 x time-step에 대하여 output을 냄)
model.add(LSTM(28, dropout=0.3, recurrent_dropout=0.3, return_sequences = True, input_shape=( TimeRange, nFeatures,  )))
model.add(LSTM(14, dropout=0.3, recurrent_dropout=0.3, return_sequences = False))
model.add(Dense(4, activation='softmax'))

In [18]:
model.compile(loss = "categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 206, 28)           3584      
                                                                 
 lstm_1 (LSTM)               (None, 14)                2408      
                                                                 
 dense_1 (Dense)             (None, 4)                 60        
                                                                 
Total params: 6,052
Trainable params: 6,052
Non-trainable params: 0
_________________________________________________________________


In [19]:
history_finetunning = model.fit(
    Train_x, Train_y,
    epochs=300,
    batch_size=52,
    callbacks=CALLBACK,
    validation_data=(Test_x, Test_y),
    shuffle=True)

Epoch 1/300
3/3 [==============================] - ETA: 0s - loss: 1.3925 - accuracy: 0.2044
Epoch 1: val_loss improved from inf to 1.38093, saving model to MODEL\001-1.3925-1.3809.hdf5
3/3 [==============================] - 4s 354ms/step - loss: 1.3925 - accuracy: 0.2044 - val_loss: 1.3809 - val_accuracy: 0.2681 - lr: 0.0010
Epoch 2/300
3/3 [==============================] - ETA: 0s - loss: 1.3832 - accuracy: 0.2701
Epoch 2: val_loss improved from 1.38093 to 1.37557, saving model to MODEL\002-1.3832-1.3756.hdf5
3/3 [==============================] - 0s 175ms/step - loss: 1.3832 - accuracy: 0.2701 - val_loss: 1.3756 - val_accuracy: 0.2681 - lr: 0.0010
Epoch 3/300
3/3 [==============================] - ETA: 0s - loss: 1.3656 - accuracy: 0.2701
Epoch 3: val_loss improved from 1.37557 to 1.37034, saving model to MODEL\003-1.3656-1.3703.hdf5
3/3 [==============================] - 0s 174ms/step - loss: 1.3656 - accuracy: 0.2701 - val_loss: 1.3703 - val_accuracy: 0.2609 - lr: 0.0010
Epoch 4/

3/3 [==============================] - ETA: 0s - loss: 0.9551 - accuracy: 0.6131
Epoch 26: val_loss improved from 1.09337 to 1.06611, saving model to MODEL\026-0.9551-1.0661.hdf5
3/3 [==============================] - 0s 170ms/step - loss: 0.9551 - accuracy: 0.6131 - val_loss: 1.0661 - val_accuracy: 0.4638 - lr: 0.0010
Epoch 27/300
3/3 [==============================] - ETA: 0s - loss: 0.9432 - accuracy: 0.5985
Epoch 27: val_loss improved from 1.06611 to 1.04505, saving model to MODEL\027-0.9432-1.0451.hdf5
3/3 [==============================] - 0s 173ms/step - loss: 0.9432 - accuracy: 0.5985 - val_loss: 1.0451 - val_accuracy: 0.4928 - lr: 0.0010
Epoch 28/300
3/3 [==============================] - ETA: 0s - loss: 0.9383 - accuracy: 0.6277
Epoch 28: val_loss improved from 1.04505 to 1.04427, saving model to MODEL\028-0.9383-1.0443.hdf5
3/3 [==============================] - 0s 171ms/step - loss: 0.9383 - accuracy: 0.6277 - val_loss: 1.0443 - val_accuracy: 0.4783 - lr: 0.0010
Epoch 29/30

3/3 [==============================] - ETA: 0s - loss: 0.6298 - accuracy: 0.7664
Epoch 52: val_loss did not improve from 0.83178
3/3 [==============================] - 0s 161ms/step - loss: 0.6298 - accuracy: 0.7664 - val_loss: 0.8853 - val_accuracy: 0.6232 - lr: 0.0010
Epoch 53/300
3/3 [==============================] - ETA: 0s - loss: 0.6073 - accuracy: 0.7883
Epoch 53: val_loss did not improve from 0.83178
3/3 [==============================] - 0s 162ms/step - loss: 0.6073 - accuracy: 0.7883 - val_loss: 0.8659 - val_accuracy: 0.6304 - lr: 0.0010
Epoch 54/300
3/3 [==============================] - ETA: 0s - loss: 0.5589 - accuracy: 0.7883
Epoch 54: val_loss did not improve from 0.83178

Epoch 54: ReduceLROnPlateau reducing learning rate to 0.000800000037997961.
3/3 [==============================] - 0s 164ms/step - loss: 0.5589 - accuracy: 0.7883 - val_loss: 0.8495 - val_accuracy: 0.6304 - lr: 0.0010
Epoch 55/300
3/3 [==============================] - ETA: 0s - loss: 0.6009 - accurac

3/3 [==============================] - 0s 163ms/step - loss: 0.7317 - accuracy: 0.7591 - val_loss: 1.0500 - val_accuracy: 0.5797 - lr: 8.0000e-04
Epoch 80/300
3/3 [==============================] - ETA: 0s - loss: 0.6792 - accuracy: 0.7737
Epoch 80: val_loss did not improve from 0.76432
3/3 [==============================] - 0s 161ms/step - loss: 0.6792 - accuracy: 0.7737 - val_loss: 0.9138 - val_accuracy: 0.6522 - lr: 8.0000e-04
Epoch 81/300
3/3 [==============================] - ETA: 0s - loss: 0.5896 - accuracy: 0.8248
Epoch 81: val_loss improved from 0.76432 to 0.74987, saving model to MODEL\081-0.5896-0.7499.hdf5
3/3 [==============================] - 0s 169ms/step - loss: 0.5896 - accuracy: 0.8248 - val_loss: 0.7499 - val_accuracy: 0.7319 - lr: 8.0000e-04
Epoch 82/300
3/3 [==============================] - ETA: 0s - loss: 0.5278 - accuracy: 0.8540
Epoch 82: val_loss improved from 0.74987 to 0.74682, saving model to MODEL\082-0.5278-0.7468.hdf5
3/3 [==============================]

3/3 [==============================] - 0s 163ms/step - loss: 0.4968 - accuracy: 0.8175 - val_loss: 0.6580 - val_accuracy: 0.7609 - lr: 8.0000e-04
Epoch 107/300
3/3 [==============================] - ETA: 0s - loss: 0.4708 - accuracy: 0.8248
Epoch 107: val_loss did not improve from 0.58807

Epoch 107: ReduceLROnPlateau reducing learning rate to 0.0006400000303983689.
3/3 [==============================] - 0s 164ms/step - loss: 0.4708 - accuracy: 0.8248 - val_loss: 0.6609 - val_accuracy: 0.7609 - lr: 8.0000e-04
Epoch 108/300
3/3 [==============================] - ETA: 0s - loss: 0.4116 - accuracy: 0.8467
Epoch 108: val_loss did not improve from 0.58807
3/3 [==============================] - 0s 163ms/step - loss: 0.4116 - accuracy: 0.8467 - val_loss: 0.6206 - val_accuracy: 0.7899 - lr: 6.4000e-04
Epoch 109/300
3/3 [==============================] - ETA: 0s - loss: 0.4173 - accuracy: 0.8613
Epoch 109: val_loss did not improve from 0.58807
3/3 [==============================] - 0s 164ms/ste

...

Epoch 189/300
3/3 [==============================] - ETA: 0s - loss: 0.2506 - accuracy: 0.9270
Epoch 189: val_loss did not improve from 0.58807
3/3 [==============================] - 0s 165ms/step - loss: 0.2506 - accuracy: 0.9270 - val_loss: 0.6303 - val_accuracy: 0.7971 - lr: 1.0737e-04
Epoch 190/300
3/3 [==============================] - ETA: 0s - loss: 0.2976 - accuracy: 0.9124
Epoch 190: val_loss did not improve from 0.58807
3/3 [==============================] - 0s 165ms/step - loss: 0.2976 - accuracy: 0.9124 - val_loss: 0.6299 - val_accuracy: 0.7971 - lr: 1.0737e-04
Epoch 191/300
3/3 [==============================] - ETA: 0s - loss: 0.2958 - accuracy: 0.9197
Epoch 191: val_loss did not improve from 0.58807
3/3 [==============================] - 0s 165ms/step - loss: 0.2958 - accuracy: 0.9197 - val_loss: 0.6292 - val_accuracy: 0.7971 - lr: 1.0737e-04
Epoch 192/300
3/3 [==============================] - ETA: 0s - loss: 0.2500 - accuracy: 0.9343
Epoch 192: val_loss did not improve

Epoch 217/300
3/3 [==============================] - ETA: 0s - loss: 0.2949 - accuracy: 0.8978
Epoch 217: val_loss did not improve from 0.58807

Epoch 217: ReduceLROnPlateau reducing learning rate to 5.497558740898967e-05.
3/3 [==============================] - 0s 162ms/step - loss: 0.2949 - accuracy: 0.8978 - val_loss: 0.6018 - val_accuracy: 0.8043 - lr: 6.8719e-05
Epoch 218/300
3/3 [==============================] - ETA: 0s - loss: 0.2656 - accuracy: 0.9051
Epoch 218: val_loss did not improve from 0.58807
3/3 [==============================] - 0s 163ms/step - loss: 0.2656 - accuracy: 0.9051 - val_loss: 0.6001 - val_accuracy: 0.8043 - lr: 5.4976e-05
Epoch 219/300
3/3 [==============================] - ETA: 0s - loss: 0.2709 - accuracy: 0.9197
Epoch 219: val_loss did not improve from 0.58807
3/3 [==============================] - 0s 164ms/step - loss: 0.2709 - accuracy: 0.9197 - val_loss: 0.5996 - val_accuracy: 0.8043 - lr: 5.4976e-05
Epoch 220/300
3/3 [==============================] 

3/3 [==============================] - ETA: 0s - loss: 0.2477 - accuracy: 0.9270
Epoch 244: val_loss did not improve from 0.58605
3/3 [==============================] - 0s 161ms/step - loss: 0.2477 - accuracy: 0.9270 - val_loss: 0.6519 - val_accuracy: 0.7899 - lr: 3.5184e-05
Epoch 245/300
3/3 [==============================] - ETA: 0s - loss: 0.2657 - accuracy: 0.9270
Epoch 245: val_loss did not improve from 0.58605
3/3 [==============================] - 0s 160ms/step - loss: 0.2657 - accuracy: 0.9270 - val_loss: 0.6519 - val_accuracy: 0.7899 - lr: 3.5184e-05
Epoch 246/300
3/3 [==============================] - ETA: 0s - loss: 0.2122 - accuracy: 0.9416
Epoch 246: val_loss did not improve from 0.58605
3/3 [==============================] - 0s 161ms/step - loss: 0.2122 - accuracy: 0.9416 - val_loss: 0.6520 - val_accuracy: 0.7899 - lr: 3.5184e-05
Epoch 247/300
3/3 [==============================] - ETA: 0s - loss: 0.2246 - accuracy: 0.9343
Epoch 247: val_loss did not improve from 0.58605


3/3 [==============================] - 0s 165ms/step - loss: 0.2297 - accuracy: 0.9343 - val_loss: 0.6520 - val_accuracy: 0.7899 - lr: 2.2518e-05
Epoch 272/300
3/3 [==============================] - ETA: 0s - loss: 0.2476 - accuracy: 0.9270
Epoch 272: val_loss did not improve from 0.58605
3/3 [==============================] - 0s 165ms/step - loss: 0.2476 - accuracy: 0.9270 - val_loss: 0.6520 - val_accuracy: 0.7899 - lr: 1.8014e-05
Epoch 273/300
3/3 [==============================] - ETA: 0s - loss: 0.2278 - accuracy: 0.9416
Epoch 273: val_loss did not improve from 0.58605
3/3 [==============================] - 0s 167ms/step - loss: 0.2278 - accuracy: 0.9416 - val_loss: 0.6520 - val_accuracy: 0.7899 - lr: 1.8014e-05
Epoch 274/300
3/3 [==============================] - ETA: 0s - loss: 0.2554 - accuracy: 0.9197
Epoch 274: val_loss did not improve from 0.58605
3/3 [==============================] - 0s 164ms/step - loss: 0.2554 - accuracy: 0.9197 - val_loss: 0.6520 - val_accuracy: 0.7899 -

3/3 [==============================] - ETA: 0s - loss: 0.2508 - accuracy: 0.9343
Epoch 299: val_loss did not improve from 0.58605
3/3 [==============================] - 0s 169ms/step - loss: 0.2508 - accuracy: 0.9343 - val_loss: 0.6245 - val_accuracy: 0.7971 - lr: 1.1529e-05
Epoch 300/300
3/3 [==============================] - ETA: 0s - loss: 0.2037 - accuracy: 0.9489
Epoch 300: val_loss did not improve from 0.58605
3/3 [==============================] - 0s 171ms/step - loss: 0.2037 - accuracy: 0.9489 - val_loss: 0.6245 - val_accuracy: 0.7971 - lr: 1.1529e-05
